In [ ]:
pip install matplotlib
pip install scikit-learn

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
test_dir = '/workspace/dataset/test'
train_dir = '/workspace/dataset/train'

# Parameters
batch_size = 8
img_height = 224
img_width = 224
seed = 123

# Load training data
train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=seed
)

# Load test data
test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=False
)

Found 76454 files belonging to 2 classes.
Found 20697 files belonging to 2 classes.


In [15]:
import tensorflow.keras.backend as K
from tensorflow.keras import layers

def preprocess_image(image, label):
    # Example: Cropping the central 224x224 region
    image = tf.image.resize_with_crop_or_pad(image, img_height, img_width)
    
    # Enhance contrast
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    
    return image, label

train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

In [17]:
# Add this before the class weight computation
AUTOTUNE = tf.data.AUTOTUNE

# Optimize the data pipeline
train_ds = train_ds.unbatch()
train_ds = train_ds.cache()  # Cache after take
train_ds = train_ds.shuffle(buffer_size=100)  # Smaller shuffle buffer
train_ds = train_ds.batch(8)  # Rebatch
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [18]:
pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sklearn.utils import class_weight
import numpy as np

try:
    print("Starting label extraction...")
    train_labels = []
    
    # Process only a subset of the dataset
    max_samples = 1000
    sample_count = 0
    
    # Disable shuffling temporarily for consistent sampling
    for images, labels in train_ds.unbatch().take(max_samples):
        label_value = labels.numpy().item()  # Convert to Python scalar
        train_labels.append(label_value)
        sample_count += 1
        
        if sample_count >= max_samples:
            break
            
    train_labels = np.array(train_labels)
    
    # Get class distribution
    unique, counts = np.unique(train_labels, return_counts=True)
    print("\nClass distribution:")
    for class_id, count in zip(unique, counts):
        print(f"Class {int(class_id)}: {count} samples")
    
    # Compute class weights with explicit type conversion
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_labels),
        y=train_labels
    )
    
    # Convert to dictionary with integer keys
    class_weights = {int(i): float(weight) for i, weight in enumerate(class_weights)}
    print("\nComputed class weights:", class_weights)

except Exception as e:
    print(f"Error during computation: {str(e)}")
    # Fallback weights - give more weight to minority class
    class_weights = {0: 7.0, 1: 1.0}  # Adjusted based on class distribution
    print("Using fallback class weights:", class_weights)

# Print final confirmation
print("\nFinal class weights to be used:", class_weights)

Starting label extraction...

Class distribution:
Class 0: 61 samples
Class 1: 939 samples

Computed class weights: {0: 8.19672131147541, 1: 0.5324813631522897}

Final class weights to be used: {0: 8.19672131147541, 1: 0.5324813631522897}


In [19]:
# 1. First, let's check class distribution
for images, labels in train_ds.take(1000):
    if 'label_counts' not in locals():
        label_counts = {}
    for label in labels.numpy():
        label_counts[float(label)] = label_counts.get(float(label), 0) + 1

print("Class distribution in training data:", label_counts)


/tmp/ipykernel_649/3430195037.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_counts[float(label)] = label_counts.get(float(label), 0) + 1
2024-10-30 10:22:44.117646: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-10-30 10:22:44.124161: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the datase

Class distribution in training data: {1.0: 9369, 0.0: 631}


In [20]:
# 1. First, calculate proper class weights
total_samples = 934 + 66
weight_for_0 = (1 / 66) * (total_samples / 2)
weight_for_1 = (1 / 934) * (total_samples / 2)

class_weights = {0: weight_for_0, 1: weight_for_1}
print("Adjusted class weights:", class_weights)

# 2. Modified model with class imbalance handling
model = tf.keras.Sequential([
    # Input layer
    tf.keras.layers.Input(shape=(img_height, img_width, 3)),
    
    # More aggressive data augmentation for minority class
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomBrightness(0.2),
    
    # Normalize pixels
    tf.keras.layers.Rescaling(1./255),
    
    # Convolutional layers with regularization
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    # Dense layers with dropout
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 3. Compile with balanced metrics


Adjusted class weights: {0: 7.575757575757576, 1: 0.5353319057815846}


In [23]:
# 3. Compile with balanced metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.F1Score(name='f1')
    ]
)

# 4. Add callbacks for better training
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_f1',
        mode='max',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_f1',
        mode='max',
        factor=0.2,
        patience=3,
        min_lr=1e-6
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_f1',
        mode='max',
        save_best_only=True
    )
]

# Calculate steps per epoch and validation steps
import math

BATCH_SIZE = 8  # Your current batch size
train_size = 76454  # From your earlier output
test_size = 20696   # From your earlier output

steps_per_epoch = math.ceil(train_size / BATCH_SIZE)
validation_steps = math.ceil(test_size / BATCH_SIZE)

# Modified training
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    steps_per_epoch=125,
    validation_steps=validation_steps,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Print training summary
print("\nTraining Summary:")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")


Epoch 1/20
123/125 [============================>.] - ETA: 0s - loss: 1.7264 - accuracy: 0.6291 - precision: 0.9633 - recall: 0.6276 - auc: 0.6924 - f1: 0.9669

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


125/125 [==============================] - 33s 229ms/step - loss: 1.7228 - accuracy: 0.6300 - precision: 0.9639 - recall: 0.6282 - auc: 0.6967 - f1: 0.9669 - val_loss: 1.3926 - val_accuracy: 0.8239 - val_precision: 0.9597 - val_recall: 0.8487 - val_auc: 0.7569 - val_f1: 0.9701 - lr: 0.0010
Epoch 2/20
125/125 [==============================] - 28s 229ms/step - loss: 1.5836 - accuracy: 0.6430 - precision: 0.9785 - recall: 0.6325 - auc: 0.7719 - f1: 0.9669 - val_loss: 1.4039 - val_accuracy: 0.8146 - val_precision: 0.9705 - val_recall: 0.8283 - val_auc: 0.8005 - val_f1: 0.9701 - lr: 0.0010
Epoch 3/20
125/125 [==============================] - 27s 220ms/step - loss: 1.5545 - accuracy: 0.6780 - precision: 0.9666 - recall: 0.6798 - auc: 0.6998 - f1: 0.9675 - val_loss: 1.8653 - val_accuracy: 0.4713 - val_precision: 0.9963 - val_recall: 0.4404 - val_auc: 0.8574 - val_f1: 0.9701 - lr: 0.0010
Epoch 4/20
125/125 [==============================] - 27s 217ms/step - loss: 1.4626 - accuracy: 0.7020 - 

2024-10-30 10:29:27.713829: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-10-30 10:29:27.719324: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



Training Summary:
Steps per epoch: 9557
Validation steps: 2587


In [24]:

# 6. Evaluate and print confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Get predictions
y_pred = []
y_true = []

for images, labels in test_ds:
    predictions = model.predict(images)
    y_pred.extend((predictions > 0.5).astype(int))
    y_true.extend(labels.numpy())

# Print detailed metrics
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

1/1 [==============================] - 0s 31ms/step


2024-10-30 10:30:51.385921: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-10-30 10:30:51.387349: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1/1 [==============================] - 0s 20ms/step


2024-10-30 10:31:10.524537: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-10-30 10:31:10.525943: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


KeyboardInterrupt: 

In [17]:
from tensorflow.keras import layers, models

# 2. Modified model with better monitoring
model = tf.keras.Sequential([
    # Input layer
    tf.keras.layers.Input(shape=(img_height, img_width, 3)),
    
    # Data augmentation
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    
    # Normalize pixels
    tf.keras.layers.Rescaling(1./255),
    
    # Convolutional layers
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    # Dense layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    class_weight=class_weights
)

Epoch 1/10
    125/Unknown 22s 168ms/step - accuracy: 0.8071 - loss: 1.0610

2024-10-30 10:00:01.711178: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


KeyboardInterrupt: 

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    class_weight=class_weights
)

In [ ]:
import matplotlib.pyplot as plt

# Evaluate the model
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Plot training & validation accuracy values
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.save('hippo_classifier_model.h5')

# 2. Convert the Model to TensorFlow Lite

In [ ]:
model = tf.keras.models.load_model('cifar10_model.h5')


In [ ]:
def representative_dataset():
    for i in range(100):
        yield [x_train[i:i+1]]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()


In [ ]:
with open('cifar10_model_quant.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
#Objective: Convert the TensorFlow Lite model into a C array to include it in the ESP-IDF project.

And then put the following in the commandline: 

xxd -i cifar10_model_quant.tflite > model.h
